# データ収集

## 公式記録から勝敗データを収集

[公式記録](https://data.j-league.or.jp/)からデータを収集する。

APIとかで取得できるようにはなっていないので、HTMLをパースして加工しやすいような形式にすることを考える。

`https://data.j-league.or.jp/SFMS01/search?competition_years=<年度>&competition_frame_ids=<カテゴリ>`

In [1]:
import requests
from bs4 import BeautifulSoup
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
year = 2018
competition = 2
url = 'https://data.j-league.or.jp/SFMS01/search?competition_years='+ str(year) + '&competition_frame_ids=' + str(competition)

print("URL= " + url)
resp = requests.get(url)
#print(resp.text)

URL= https://data.j-league.or.jp/SFMS01/search?competition_years=2018&competition_frame_ids=2


HTMLを直接見ると、tableタグのデータを見れば良さそう。

Tableタグの取得は、pandasで簡単に取れるらしい([参考](https://qiita.com/kitsuyui/items/4906bb457af4d0e2d0a5))ので、これを使う。

In [3]:
import pandas as pd
from IPython.core.display import display
dfs = pd.io.html.read_html(url)
dfs[0] = dfs[0].drop(['インターネット中継・TV放送','入場者数'],axis=1) # 結果に関係なさそうな放送局と入場者数はこの時点で削除しておく
display(dfs[0])

,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム
0,2018,Ｊ２,第１節第１日,02/25(日),13:04,福岡,2-0,岐阜,レベスタ
1,2018,Ｊ２,第１節第１日,02/25(日),14:03,栃木,2-4,大分,栃木グ
2,2018,Ｊ２,第１節第１日,02/25(日),14:03,愛媛,1-2,金沢,ニンスタ
3,2018,Ｊ２,第１節第１日,02/25(日),14:03,東京Ｖ,2-1,千葉,味スタ
4,2018,Ｊ２,第１節第１日,02/25(日),14:03,讃岐,0-1,新潟,ピカスタ
...,...,...,...,...,...,...,...,...,...
457,2018,Ｊ２,第４２節第１日,11/17(土),14:03,金沢,3-1,水戸,石川西部
458,2018,Ｊ２,第４２節第１日,11/17(土),14:03,岡山,0-1,大宮,Ｃスタ
459,2018,Ｊ２,第４２節第１日,11/17(土),14:03,甲府,0-1,横浜FC,中銀スタ
460,2018,Ｊ２,第４２節第１日,11/17(土),14:03,岐阜,0-0,福岡,長良川


ここまでで、[download.py](download.py)の関数download_resultを作成したので、これで試してみる。

In [4]:
import download
df = download.download_result(year=2021)
display(df)

,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,入場者数,インターネット中継・TV放送
0,2021,ＦＵＪＩ ＸＥＲＯＸ ＳＵＰＥＲ ＣＵＰ,NaN,02/20(土),13:35,川崎Ｆ,3-2,Ｇ大阪,埼玉,4208.0,日本テレビ系全国ネットにて生中継
1,2021,Ｊ１,第１節第１日,02/26(金),18:04,川崎Ｆ,2-0,横浜FM,等々力,4868.0,ＤＡＺＮ
2,2021,Ｊ１,第１節第２日,02/27(土),14:00,浦和,1-1,FC東京,埼玉,4943.0,ＤＡＺＮ／ＮＨＫ総合
3,2021,Ｊ１,第１節第２日,02/27(土),14:00,札幌,5-1,横浜FC,札幌ド,11897.0,ＤＡＺＮ／ＮＨＫ札幌
4,2021,Ｊ１,第１節第２日,02/27(土),14:03,大分,1-1,徳島,昭和電ド,7012.0,ＤＡＺＮ／ＮＨＫ大分／ＮＨＫ徳島／ＮＨＫ松山／ＮＨＫ高知／ＮＨＫ高松
...,...,...,...,...,...,...,...,...,...,...,...
1394,2021,Ｊユースリーグ グループＪ,第７節,未定,NaN,熊本,vs,北九州,●未定●,NaN,NaN
1395,2021,Ｊユースリーグ グループＪ,第７節,未定,NaN,鹿児島,vs,長崎,●未定●,NaN,NaN
1396,2021,Ｊユースリーグ グループＪ,第８節,未定,NaN,宮崎,vs,長崎,●未定●,NaN,NaN
1397,2021,Ｊユースリーグ グループＪ,第９節,未定,NaN,鳥栖,vs,鹿児島,●未定●,NaN,NaN


CSVとかでいいけど、最近データベースを勉強中なのでsqlite3に保存してみる  
pandasのDataFrameはto_sqlメソッドでSQLに変換可能らしい([参考](http://www.stockdog.work/entry/2017/03/22/010057))。

保存場所として、data/jleague.dbとするものとした。  
この場合、dataフォルダが存在しない場合は作ることにする。

In [5]:
import sqlite3
import os
db_name = "data/test.db"
if os.path.exists('data') == False:
    os.mkdir('data')
conn = sqlite3.connect(db_name) # DB接続
dfs[0].to_sql("result", conn, if_exists='replace') # DFをDBに変換

462

ここまでで、download_all_resultsにした。

download_all_resultsは、1992年～2017年のデータを取得し、sqlite(data/jleague.db)に変換する。  
jleague.dbのresultというデータベースに、取得した結果を格納する。

In [6]:
download.download_all_results(end=2022)

生成されたDBを見てみる。

In [7]:
import sqlite3
import pandas as pd
db_name = 'data/jleague.db'
conn = sqlite3.connect(db_name)
df = pd.read_sql("select * from result", conn)
conn.close()

display(df)

,index,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,入場者数,インターネット中継・TV放送
0,0,1992,ＹＮＣ,第１節第１日,09/05(土),15:00,鹿島,4-2,横浜Ｆ,笠松,5226.0,None
1,1,1992,ＹＮＣ,第１節第１日,09/05(土),15:30,名古屋,3-0,清水,瑞穂球,8029.0,None
2,2,1992,ＹＮＣ,第１節第１日,09/05(土),18:02,浦和,2-3,市原,大宮,4934.0,None
3,3,1992,ＹＮＣ,第１節第１日,09/05(土),19:00,Ｇ大阪,2-0,横浜M,神戸中央,4728.0,None
4,4,1992,ＹＮＣ,第１節第２日,09/06(日),19:00,広島,2-3,Ｖ川崎,広島ス,13861.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...
22902,1394,2021,Ｊユースリーグ グループＪ,第７節,未定,None,熊本,vs,北九州,●未定●,NaN,None
22903,1395,2021,Ｊユースリーグ グループＪ,第７節,未定,None,鹿児島,vs,長崎,●未定●,NaN,None
22904,1396,2021,Ｊユースリーグ グループＪ,第８節,未定,None,宮崎,vs,長崎,●未定●,NaN,None
22905,1397,2021,Ｊユースリーグ グループＪ,第９節,未定,None,鳥栖,vs,鹿児島,●未定●,NaN,None


全18118レコード取得可能。

## Football-Labからデータを収集

[Football-Lab](http://www.football-lab.jp/)はJリーグのデータ分析で有名なデータスタジアムが運営するWebサイト。  
勝敗データだけでなく、得点がどんな得点だったか/チームとして志向するスタイル等が見えるデータを提供している。

### 提供しているデータ

提供されているデータの中から、チーム毎のデータとして、シーズン毎の
* Team Data
  * 順位表
  * チャンス構築率
  * 得点パターン
  * 失点パターン
  * ボール支配率
* チームスタイル指標
  * ショートカウンター
  * ロングカウンター
  * 敵陣ポゼッション
  * 自陣ポゼッション
  * 左サイド攻撃
  * 中央攻撃
  * 右サイド攻撃
* チャンスビルディングポイント
  * 攻撃
  * パス
  * ドリブル
  * クロス
  * シュート
  * 守備
  * セーブ

を収集してみる。

### Team Data: 順位表

現時点の順位表データを取得する。

In [8]:
import pandas as pd
def get_rank_table(competition=1, year=2018):
    url = "http://www.football-lab.jp/summary/team_ranking/j" + str(competition) + "/?year=" + str(year)
    dfs = pd.io.html.read_html(url)
    df = dfs[0].drop("Unnamed: 1",axis=1).rename(columns={"Unnamed: 2":"チーム"})
    return df

In [9]:
rank = get_rank_table(competition=1, year=2017)
#display(rank)

チーム名がおかしいが、省略しないチーム名+省略名(公式データの省略名と同じ)になっているようなので、
公式データのチーム名と比較することにする。

In [10]:
import download
df = download.get_db_data()

In [11]:
teams = pd.concat([df["ホーム"],df["アウェイ"]]).drop_duplicates()
#display(teams)

In [12]:
for i in range(len(rank["チーム"])):
    for team in teams:
        if rank.loc[i, "チーム"].endswith(team) is True:
            rank.loc[i, "チーム"] = team
            print(rank["チーム"].iloc[i])

川崎Ｆ
鹿島
Ｃ大阪
柏
横浜FM
磐田
浦和
鳥栖
神戸
Ｇ大阪
札幌
仙台
FC東京
清水
広島
甲府
新潟
大宮


In [13]:
import download
rank = download.fl_get_rank(1,2018)
#display(rank)

### Team Data:チャンス構築率

|データ項目|プレー定義|
|----------|----------|
|攻撃(回数)|ある特定の状況において例外はあるものの、ボールを保持してから相手チームに渡る、<br>もしくはファウルやボールアウトで試合が止まるまでの間を１回の攻撃とする|
|シュート|データスタジアムが独自に集計したシュート数であり、公式記録の数値とは異なる|
|チャンス構築率|上記のシュート数÷攻撃回数により算出|
|ゴール|得点となったシュートの数。オウンゴールは含まれない|
|シュート成功率|上記のゴール数÷シュート数により算出|

In [14]:
url = "http://www.football-lab.jp/summary/team_ranking/j1/?year=2018&data=chance"
dfs = pd.io.html.read_html(url)
#print(dfs)

In [15]:
def get_chance_rate(competition=1, year=2018):
    url = "http://www.football-lab.jp/summary/team_ranking/j" + str(competition) + "/?year=" + str(year) + "&data=chance"
    dfs = pd.io.html.read_html(url)
    attack = dfs[0].drop("Unnamed: 0",axis=1)
    attack = attack.drop("Unnamed: 3",axis=1)
    attack = attack.drop("Unnamed: 5",axis=1)
    attack = attack.drop("Unnamed: 7",axis=1)
    attack = attack.drop("Unnamed: 9",axis=1)
    attack = attack.drop("Unnamed: 11",axis=1)
    attack = attack.rename(columns={"Unnamed: 1":"チーム"})
    defense = dfs[2].drop("Unnamed: 0",axis=1)
    defense = defense.drop("Unnamed: 3",axis=1)
    defense = defense.drop("Unnamed: 5",axis=1)
    defense = defense.drop("Unnamed: 7",axis=1)
    defense = defense.drop("Unnamed: 9",axis=1)
    defense = defense.drop("Unnamed: 11",axis=1)
    defense = defense.rename(columns={"Unnamed: 1":"チーム"})
    return attack, defense

In [16]:
atk, dfe = get_chance_rate()
#display(atk)

In [17]:
atk, dfe = download.fl_get_chance_rate(competition=1,year=2018)
#display(atk)

### Team Data: ゴールパターン&失点パターン

テーブル属性ではないので、Beautifulsoupで収集。

In [18]:
from pprint import pprint
import requests
import re
import ast
from bs4 import BeautifulSoup,Comment

url = "http://www.football-lab.jp/summary/team_ranking/j1/?year=2018&data=goal"

response = requests.get(url)
bs = BeautifulSoup(response.content,"lxml")
test=str(bs.find(string=re.compile("function drawChart")))

tbl = re.search(r'arrayToDataTable\((.*?)\)', test, flags=re.DOTALL|re.MULTILINE).group(1)
pat = pd.DataFrame(ast.literal_eval(tbl)[1:],columns=ast.literal_eval(tbl)[0])
#display(pat)

In [19]:
pat = download.fl_get_goal_pattern(competition=2, year=2021)
display(pat)

,チーム,ＰＫ,セットプレー直接,セットプレーから,クロスから,スルーパスから,ショートパスから,ロングパスから,ドリブルから,こぼれ球から,その他
0,磐田,2,1,15,23,8,11,5,2,7,1
1,長崎,2,0,16,17,8,11,2,4,7,2
2,甲府,3,1,18,12,4,14,1,4,5,3
3,町田,3,1,14,12,3,14,4,3,8,2
4,東京Ｖ,3,1,7,15,3,20,0,4,7,2
5,新潟,3,1,11,14,5,9,1,6,8,3
6,山形,2,1,17,14,6,9,0,4,5,3
7,水戸,4,0,7,9,14,8,1,10,4,2
8,京都,2,0,8,14,4,15,3,3,7,3
9,琉球,3,2,12,20,2,10,1,1,5,1


### Team Data: ボール支配率


In [20]:
url = "http://www.football-lab.jp/summary/team_ranking/j1/?year=2017&data=possession"
response = requests.get(url)
bs = BeautifulSoup(response.content,"lxml")
test=str(bs.find(string=re.compile("function drawChart"))).replace("(%)","%")    # なぜか(%)のところが変換できないので文字列として変換
tbl = re.search(r'arrayToDataTable\((.*?)\)', test, flags=re.DOTALL|re.MULTILINE).group(1)
tbl = re.sub(", \{.+?\}","", tbl)   # annotation属性が残ってしまうので削除
col = ast.literal_eval(tbl)[0]
col.append("dummy")
pos = pd.DataFrame(ast.literal_eval(tbl)[1:],columns=col).drop("dummy",axis=1) # 不必要な列があるのでdummyと命名して削除
#display(pos)

In [21]:
pos = download.fl_get_possession(competition=2,year=2018)
#display(pos)

### チャンスビルディングポイント

URLは`http://www.football-lab.jp/summary/cbp_ranking/j1/?year=<YEAR>&data=<DATA>`」で、

|データ|<DATA>|
|-----|-------|
|攻撃|なし|
|パス|pass|
|ドリブル|dribble|
|クロス|cross|
|シュート|shot|
|守備|defense|
|セーブ|save|

という差。
* チーム名がUnnamed: 2になっていて、Unnnamed: 0-1は不要。
* 順位表同様、チーム名はリネームする。
* 順位・勝ち点・得点・失点は別のデータにあるので削除しておく

In [22]:
dfs = pd.io.html.read_html("http://www.football-lab.jp/summary/cbp_ranking/j1/?year=2013")
#display(dfs[0])

In [23]:
cbp = download.fl_get_cbp(competition=2, year=2022, data="dribble")
#display(cbp)

### まとめ

ここまでのデータを、チーム名でまとめてデータフレーム化する。

得点パターンと失点パターンが一緒なので、renameする。

In [24]:
rank = download.fl_get_rank(1,2018)
atk, dfe = download.fl_get_chance_rate(competition=1,year=2018)
goal_pat = download.fl_get_goal_pattern(competition=1, year=2018)
goal_pat = goal_pat.rename(columns={"ＰＫ":"ＰＫ_得点",
                                    "セットプレー直接":"セットプレー直接_得点",
                                    "セットプレーから":"セットプレーから_得点",
                                    "クロスから":"クロスから_得点",
                                    "スルーパスから":"スルーパスから_得点",
                                    "ショートパスから":"ショートパスから_得点",
                                    "ロングパスから":"ロングパスから_得点",
                                    "ドリブルから":"ドリブルから_得点",
                                    "こぼれ球から":"こぼれ球から_得点",
                                    "その他":"その他_得点"
                                   })
lost_pat = download.fl_get_lost_pattern(competition=1, year=2018)
lost_pat = lost_pat.rename(columns={"ＰＫ":"ＰＫ_失点",
                                    "セットプレー直接":"セットプレー直接_失点",
                                    "セットプレーから":"セットプレーから_失点",
                                    "クロスから":"クロスから_失点",
                                    "スルーパスから":"スルーパスから_失点",
                                    "ショートパスから":"ショートパスから_失点",
                                    "ロングパスから":"ロングパスから_失点",
                                    "ドリブルから":"ドリブルから_失点",
                                    "こぼれ球から":"こぼれ球から_失点",
                                    "その他":"その他_失点"
                                   })
posession = download.fl_get_possession(competition=1,year=2018)

#fl_data = pd.merge(rank, atk, dfe, goal_pat, lost_pat, posession)
fl_data = pd.merge(rank, atk, on="チーム")
fl_data = pd.merge(fl_data, dfe, on="チーム")
fl_data = pd.merge(fl_data, goal_pat, on="チーム")
fl_data = pd.merge(fl_data, lost_pat, on="チーム")
fl_data = pd.merge(fl_data, posession, on="チーム")
fl_data["年度"] = 2018
fl_data["ディビジョン"] = "J" + str(1)
#display(fl_data)

## 使い方

``` python
download.download_all_fl_data()
```
を実施すると、DBのdataテーブルに保存される。

実施後は、
``` python
data = download.get_db_data(data="data")
```
で取得できる。

In [25]:
import download
data = download.download_all_fl_data()
#display(data)

In [26]:
data = download.get_db_data(data="data")
display(data)

,index,順位,チーム,勝点,試合数,勝,分,敗,得点,失点,...,攻撃ポイント,パスポイント,ドリブルポイント,クロスポイント_x,クロスポイント_y,シュートポイント,守備ポイント,セーブポイント,年度,ディビジョン
0,0,1,広島,64,34,19,7,8,63,34,...,539.55,416.38,54.35,68.82,68.82,285.51,493.33,16.26,2012,J1
1,1,2,仙台,57,34,15,12,7,59,43,...,470.22,350.38,39.21,80.63,80.63,287.47,512.59,12.66,2012,J1
2,2,3,浦和,55,34,15,10,9,47,42,...,578.34,453.60,61.11,63.63,63.63,269.15,510.23,12.78,2012,J1
3,3,4,横浜FM,53,34,13,14,7,44,33,...,521.81,391.53,60.51,69.77,69.77,261.03,463.31,10.54,2012,J1
4,4,5,鳥栖,53,34,15,8,11,48,39,...,361.84,282.70,24.35,54.79,54.79,238.62,556.81,15.87,2012,J1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,17,18,秋田,28,26,7,7,12,22,35,...,249.86,185.15,20.25,44.46,44.46,149.26,409.98,10.82,2022,J2
438,18,19,岩手,27,26,7,6,13,22,42,...,274.33,205.61,24.12,44.60,44.60,139.11,374.56,11.34,2022,J2
439,19,20,群馬,23,26,6,5,15,20,33,...,281.65,235.11,13.88,32.66,32.66,113.29,351.50,11.86,2022,J2
440,20,21,大宮,23,26,5,8,13,26,42,...,305.23,248.43,27.43,29.38,29.38,124.49,395.62,13.32,2022,J2
